In [1]:
#!pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu116

In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [4]:

import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor

from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations

#from math import ceil


In [5]:
from __future__ import print_function, division
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy

print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [6]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Convert to tensor and scale to [0, 1]
ts = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0,), (1,))])
mnist_trainset = datasets.MNIST('../data', train=True, download=True, transform=ts)
mnist_testset = datasets.MNIST(root='../data', train=False, download=True, transform=ts)

In [7]:
from torch.utils.data import Subset
train_subset_size = int(0.1 * len(mnist_trainset))
test_subset_size = int(0.0999 * len(mnist_testset))
train_indices = list(range(len(mnist_trainset)))
test_indices = list(range(len(mnist_testset)))
torch.manual_seed(10)

seed_value = 10
np.random.seed(seed_value)

np.random.shuffle(train_indices)
np.random.shuffle(test_indices)

train_subset_indices = train_indices[:train_subset_size]
test_subset_indices = test_indices[:test_subset_size]

mnist_trainset = Subset(mnist_trainset, train_subset_indices)
mnist_testset = Subset(mnist_testset, test_subset_indices)

In [8]:
x_d0 = mnist_trainset[0][0].size()[0]
x_d1 = mnist_trainset[0][0].size()[1]
x_d2 = mnist_trainset[0][0].size()[2]
# ([1, 28, 28])
N = x_d3 = len(mnist_trainset)
K = 10
x_train = torch.empty((N,x_d0*x_d1*x_d2), device=device)
# (60000, 28*28)

y_train = torch.empty(N, dtype=torch.long)

#torch.empty() return a tensor filled with garbage values Dimention: N by x_d0*x_d1*x_d2

for i in range(N):
     x_train[i,:] = torch.reshape(mnist_trainset[i][0], (1, x_d0*x_d1*x_d2))
     y_train[i] = mnist_trainset[i][1]
x_train = torch.t(x_train)
y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

# Manipulate test set
N_test = x_d3_test = len(mnist_testset)
x_test = torch.empty((N_test,x_d0*x_d1*x_d2), device=device)
y_test = torch.empty(N_test, dtype=torch.long)
for i in range(N_test):
     x_test[i,:] = torch.reshape(mnist_testset[i][0], (1, x_d0*x_d1*x_d2))
     y_test[i] = mnist_testset[i][1]
x_test = torch.t(x_test)
y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [9]:
#### Reshape X to X-bar
x_trainTensor = torch.reshape(x_train, (x_d1, x_d2, x_d0,-1))
x_testTensor  = torch.reshape(x_test, (x_d1, x_d2, x_d0,-1))
print(x_trainTensor.shape)
print(x_testTensor.shape)

torch.Size([28, 28, 1, 6000])
torch.Size([28, 28, 1, 999])


In [10]:
#### Chenyang: you need to write a function to transform the x_trainTensor, x_testTensor to (H'W')*(l^2C)*n
#### follow the paper in https://arxiv.org/pdf/1611.03214.pdf   this paper donot have the stride, we should also include stride
#### how to make it faster;  make sure the code is correct?
def inputX_CNN(x_Tensor, filter_size, stride):
    H, W, C, n = x_Tensor.size()
    Hprime = torch.floor(torch.tensor((H-filter_size)/stride))+1
    Hprime = Hprime.to(torch.int)
    Wprime = torch.floor(torch.tensor((W-filter_size)/stride))+1
    Wprime = Wprime.to(torch.int)
    Xtranform = torch.zeros((Hprime * Wprime, filter_size * filter_size * C, n), device=device)
    for i in range(n):
      Data =  x_Tensor[:,:,:,i]
      for hh in range(Hprime):
        for ww in range(Wprime):
          #  print(range(ww * stride, ww * stride + filter_size))
            DataTemp = Data[range(hh * stride, hh * stride + filter_size), :,:]
            DataTemp = DataTemp[:, range(ww * stride, ww * stride + filter_size), :]
            Xtranform[Hprime * hh + ww, :, i] = torch.reshape(DataTemp, (1, filter_size * filter_size * C))
    return Xtranform

In [11]:
### from X-bar to X-bar-bar
filter_size=3
stride = 2
x_trainTS = inputX_CNN(x_trainTensor,filter_size,stride)
x_testTS  = inputX_CNN(x_testTensor,filter_size,stride)

In [12]:
#### To-do-list
#### Chenyang: you need to write a function to update W for CNN
def updateWb_CNN(U, V, W, W_tensor_rec, alpha, rho,tau): # W:previous W
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)  #w shape need to take a look at
    d,N = V.size()
    I = torch.eye(N, device=device)
    U_prime = torch.t(U).reshape(n*Hprime*Wprime,-1)
    # _, col_U = U_prime.size()
    Wstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(V),V))+(alpha+tau)*I), rho*torch.mm(torch.t(V),U_prime)+alpha*W+tau*W_tensor2matrix)
    # bstar = 0
    return Wstar
# def updateWb_CNN(U, V, W, W_tensor_rec, alpha, rho,tau): # W:previous W, v for the first layer should be X-bar-bar-prime
#     W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
#     W_tensor2matrix = W_tensor_rec.reshape(W.shape)  #w shape need to take a look at
#     d,N = V.size()
#     I = torch.eye(N, device=device)
#     U_prime = torch.t(U).reshape(n*Hprime*Wprime,-1)
#     A = rho*torch.mm(torch.t(V),U_prime)+alpha*W+tau*W_tensor2matrix
#     A = torch.t(A)
#     # _, col_U = U_prime.size()
#     Wstar_transpose = torch.mm(A, torch.inverse(rho*(torch.mm(torch.t(V), V))+(alpha+tau)*I))
#     Wstar =  torch.t(Wstar_transpose)
#     return Wstar

In [13]:
def updateWb_CNNorg(U, V, W, alpha, rho):
    d,N = V.size()
    I = torch.eye(N, device=device)
    #_, col_U = U.size()
    U_prime = torch.t(U).reshape(n*Hprime*Wprime,-1)
    Wstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(V),V))+alpha*I), rho*torch.mm(torch.t(V),U_prime)+alpha*W)
    
    return Wstar

In [14]:
def updateV(U1,U2,W,b,rho,gamma):
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

In [15]:
def updateWb_org(U, V, W, b, alpha, rho):
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

In [16]:
def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau):
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

In [17]:
# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [ ]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('/home/c/cl237/TenBCD/CNN/parameter tuning/CNNmnist_hyperparameter_test.csv')
niter = 2000

#alpha = 5
#alpha1 = alpha2 = alpha3 = alpha4 = alpha5 = alpha6 = alpha7 = alpha8 = alpha9 = alpha10 = alpha

# for rank in (35,): #(25,30,35)(100,180,220,260,300,340,380)(20,60,100,140,180,220,260,300,340,380)
#     for tau in (400,500): #(300,400,500)(10,50,100,200,300)(10,50,100,200,300)
#         for gamma in (0.5,0.8,2): #(0.5,0.8,2)(0.5,0.8)(0.5,1,1.5,2,3)
#             #gamma1 = gamma2 = gamma3 = gamma4 = gamma5 = gamma
#             for rho in (0.5,0.8,2): #(0.5,0.8)(1,2)
#                 #rho1 = rho2 = rho3 = rho4 = rho5= rho
#                 for alpha in (0.5,1,1.5,2):

for rank in (40,80,140): 
    for tau in (0.1,):
        for alpha in (1,):
            for gamma in (5,): #for gamma in (0.01,0.1,0.5,1,5,10,100):
                for rho in (5,): #for rho in (0.01,0.1,0.5,1,5,10,100):

                    print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)
                    
                    loss1 = np.empty(niter)
                    loss2 = np.empty(niter)
                    accuracy_train = np.empty(niter)
                    accuracy_test = np.empty(niter)
                    time1 = np.empty(niter)

                    results = torch.zeros(10, 5, niter)

                    S = 32 ### number of filters 2^5
                    H, W, C, n = x_trainTensor.size()   # n is the same thing as N
                    Hprime = torch.floor(torch.tensor((H-filter_size)/stride))+1
                    Hprime = Hprime.to(torch.int)
                    Wprime = torch.floor(torch.tensor((W-filter_size)/stride))+1
                    Wprime = Wprime.to(torch.int)

                    

                    for Out_iter in range(10):
                        rank_initial = 700
                        seed = 10 + 10*Out_iter
                        torch.manual_seed(seed)
                        if torch.cuda.is_available():
                            torch.manual_seed(seed)

                        ## Chenyang
                        d1 =  Hprime * Wprime * S
                        d2 =  1024
                        d3 =  1024
                        d4 =  10


                        W1 = 0.01*torch.randn(filter_size * filter_size * C, S, device=device)  ## How people usually initialize the CNN kernel?
                        #W1_torch_tensor = W1.reshape((filter_size, filter_size, C, 2,2,2,2,2)) ## TBD
                        #W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
                        #factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, 1))
                        #W1_tl_tensor_rec = tt_to_tensor(factors1)
                        b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


                        W2 = 0.01*torch.randn(d2, d1, device=device)
                        W2_torch_tensor = W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2)) # 2^10 and Hprime * Wprime  *2^5*1^3
                        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
                        factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
                        # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
                        # factors: set of tensor cores
                        # http://tensorly.org/stable/user_guide/tensor_decomposition.html
                        # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
                        W2_tl_tensor_rec = tt_to_tensor(factors2)
                        # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
                        #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray
                        b2 = 0*torch.ones(d2, 1, device=device)


                        W3 = 0.01*torch.randn(d3, d2, device=device)
                        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
                        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
                        factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
                        W3_tl_tensor_rec = tt_to_tensor(factors3)
                        b3 = 0*torch.ones(d3, 1, device=device)

                        W4 = 0.01*torch.randn(d4, d3, device=device)
                        b4 = 0*torch.ones(d4, 1, device=device)

                        ## Chenyang: this is for the initialization of U1
                        x_trainTS1 = torch.reshape(x_trainTS, (-1,n))
                        x_trainTS1 = torch.t(x_trainTS1)
                        x_trainTS1 = torch.reshape(x_trainTS1, (-1, filter_size * filter_size * C))   ### this is X-bar-bar'
                        U1prime = torch.matmul(x_trainTS1, W1)
                        U1prime = torch.reshape(U1prime, (n,-1))
                        U1 = torch.t(U1prime)

                        V1 = nn.ReLU()(U1)
                        U2 = torch.addmm(b2.repeat(1, N), W2, V1)
                        V2 = nn.ReLU()(U2)
                        U3 = torch.addmm(b3.repeat(1, N), W3, V2)
                        V3 = nn.ReLU()(U3)
                        U4 = torch.addmm(b4.repeat(1, N), W4, V3)
                        V4 = U4

                        ### to create the X-bar-bar' for test data
                        x_testTS1 = torch.reshape(x_testTS, (-1, N_test))
                        x_testTS1 = torch.t(x_testTS1)
                        x_testTS1 = torch.reshape(x_testTS1, (-1, filter_size * filter_size * C))   ### this is X-bar-bar' test


                        # Iterations
                        print('Train on', N, 'samples, validate on', N_test, 'samples')
                        for k in range(niter):
                            start = time.time()

                      # update for last layer
                            # update V4
                            V4 = (y_one_hot + gamma*U4 + alpha*V4)/(1 + gamma + alpha)

                            # update U4
                            U4 = (gamma*V4 + rho*(torch.mm(W4,V3) + b4.repeat(1,N)))/(gamma + rho)

                            # update W4 and b4
                            W4, b4 = updateWb_org(U4,V3,W4,b4,alpha,rho)
                            # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
                            # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
                            # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
                            # W3_tl_tensor_rec = tt_to_tensor(factors3)


                      # update for 3nd layer
                            # update V3
                            V3 = updateV(U3,U4,W4,b4,rho,gamma)

                            # update U3
                            U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

                            # update W3 and b3
                            W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

                            # G update (TTD)
                            W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
                            W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
                            factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
                            #set of tensor cores
                            W3_tl_tensor_rec = tt_to_tensor(factors3)


                      # update for 2nd layer
                            # update V2
                            V2 = updateV(U2,U3,W3,b3,rho,gamma)

                            # update U2
                            U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

                            # update W2 and b2
                            W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

                            # G update (TTD)
                            W2_torch_tensor = W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2))
                            W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
                            factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
                            W2_tl_tensor_rec = tt_to_tensor(factors2)


                     # update for 1st layer
                            # update V1
                            V1 = updateV(U1,U2,W2,b2,rho,gamma)

                            ## Chenyang
                            XprimeW = torch.reshape(torch.matmul(x_trainTS1, W1), (n,-1))
                            XprimeWtranspose = torch.t(XprimeW)
                            # update U1
                            U1 = relu_prox(V1,(rho*XprimeWtranspose + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

                            # update W1 and b1
                            ## Chenyang: you need to use the new function updateWb_CNN
                            W1 = updateWb_CNNorg(U1,x_trainTS1,W1,alpha,rho)

                            # G update
                            #W1_torch_tensor = W1.reshape((filter_size, filter_size, C, 2,2,2,2,2))
                            #W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
                            #factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,1))
                            #W1_tl_tensor_rec = tt_to_tensor(factors1)




                      # prediction for trainning data
                            ### we should use reconstructed tensnor W1 for prediction
                            XprimeW = torch.reshape(torch.matmul(x_trainTS1, torch.as_tensor(W1,device=device).reshape((filter_size * filter_size * C, S)).float()), (n,-1))
                            XprimeWtranspose = torch.t(XprimeW)
                            a1_train = nn.ReLU()(XprimeWtranspose)
                            #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
                            a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
                            a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
                            #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
                            pred = torch.argmax(torch.addmm(b4.repeat(1, N), W4, a3_train), dim=0)
                            # check argmax and addmm, dim=0

                      #Prediction for test data
                            XprimeWtest = torch.reshape(torch.matmul(x_testTS1, torch.as_tensor(W1,device=device).reshape((filter_size * filter_size * C, S)).float()), (N_test,-1))
                            XprimeWtesttranspose = torch.t(XprimeWtest)
                            a1_test = nn.ReLU()(XprimeWtesttranspose) ## Chenyang
                            #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
                            a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
                            a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
                            pred_test = torch.argmax(torch.addmm(b4.repeat(1, N_test), W4, a3_test), dim=0)


                        #emperical loss
                            loss1[k] = gamma/2*torch.pow(torch.dist(V4,y_one_hot,2),2).cpu().numpy()

                            # Eq (5) in paper
                            ## Chenyang
                            loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(XprimeWtranspose,U1,2),2).cpu().numpy() \
                            +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
                            +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
                            +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
                            + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
                            + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
                            + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
                            + gamma/2*torch.pow(torch.dist(V4,U4,2),2).cpu().numpy() \
                            +tau/2*torch.pow(torch.dist(W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
                            +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
                            # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy()
                            # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy()

                            # compute training accuracy
                            correct_train = pred == y_train
                            accuracy_train[k] = np.mean(correct_train.cpu().numpy())

                            # compute validation accuracy
                            correct_test = pred_test == y_test
                            accuracy_test[k] = np.mean(correct_test.cpu().numpy())

                            # compute training time
                            stop = time.time()
                            duration = stop - start
                            time1[k] = duration

                            # print results
                            print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n',
                                  '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k],
                                  '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

                    ##############
                    ############## compute CR
                        # factors1_shape=[f.shape for f in factors1]
                        # Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
                        factors2_shape=[f.shape for f in factors2]
                        Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
                        factors3_shape=[f.shape for f in factors3]
                        Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
                        total_variabels=Sum_of_variables_factors2+Sum_of_variables_factors3
                        
                        layer2_CR = Sum_of_variables_factors2/(d1*d2).item()
                        layer3_CR = Sum_of_variables_factors3/(d2*d3)
                        Compressedlayers_CR = total_variabels/(d1*d2+d2*d3).item()
                        Compressedlayers_CR2 = (total_variabels+d3*d4)/(d1*d2+d2*d3+d3*d4).item()
                        
                        # print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


                        results[Out_iter,0,:] = torch.tensor(loss1)
                        results[Out_iter,1,:] = torch.tensor(loss2)
                        results[Out_iter,2,:] = torch.tensor(accuracy_train)
                        results[Out_iter,3,:] = torch.tensor(accuracy_test)
                        results[Out_iter,4,:] = torch.tensor(time1)
                        CR=(layer2_CR,layer3_CR,Compressedlayers_CR,Compressedlayers_CR2)

                        #this postion to add new row into existing table
#                         df=pd.read_csv('/home/c/cl237/TenBCD/CNN/parameter tuning/CNNmnist_hyperparameter_test.csv')
#                         new_row = {'rank':rank, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
#                                    'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
#                                    'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1], 'layer2_CR':layer2_CR, \
#                                   'layer3_CR':layer3_CR, 'Compressedlayers_CR':Compressedlayers_CR}
#                         df=df.append(new_row,ignore_index=True)
#                         df.to_csv('/home/c/cl237/TenBCD/CNN/parameter tuning/CNNmnist_hyperparameter_test.csv',index=False)


                    filename= "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
                    "rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
                    from scipy.io import savemat
                    %cd /home/c/cl237/TenBCD/CNN/same set V2 10times/
                    savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})


rank= 40 tau= 0.1 gamma= 5 rho= 5 alpha 1
Train on 6000 samples, validate on 999 samples
Repeatition 1 Epoch 1 / 2000 
 - time: 8.008145809173584 - sq_loss: 11021.754150390625 - tot_loss: 11157.073842114065 - acc: 0.14316666666666666 - val_acc: 0.14314314314314314
Repeatition 1 Epoch 2 / 2000 
 - time: 7.983645439147949 - sq_loss: 9261.334228515625 - tot_loss: 9412.726615316351 - acc: 0.18933333333333333 - val_acc: 0.19019019019019018
Repeatition 1 Epoch 3 / 2000 
 - time: 7.9843339920043945 - sq_loss: 8135.1959228515625 - tot_loss: 8242.652784969798 - acc: 0.192 - val_acc: 0.17917917917917917
Repeatition 1 Epoch 4 / 2000 
 - time: 7.827265024185181 - sq_loss: 6998.824462890625 - tot_loss: 7086.335998852504 - acc: 0.20383333333333334 - val_acc: 0.19019019019019018
Repeatition 1 Epoch 5 / 2000 
 - time: 7.8716936111450195 - sq_loss: 5991.5008544921875 - tot_loss: 6064.809410300758 - acc: 0.2285 - val_acc: 0.22522522522522523
Repeatition 1 Epoch 6 / 2000 
 - time: 7.846643924713135 - sq_

Repeatition 1 Epoch 49 / 2000 
 - time: 7.953994512557983 - sq_loss: 5.3026604652404785 - tot_loss: 6.323292911011959 - acc: 0.5125 - val_acc: 0.5325325325325325
Repeatition 1 Epoch 50 / 2000 
 - time: 7.869805574417114 - sq_loss: 4.525216519832611 - tot_loss: 5.520508782671823 - acc: 0.521 - val_acc: 0.5415415415415415
Repeatition 1 Epoch 51 / 2000 
 - time: 7.9986937046051025 - sq_loss: 3.862958252429962 - tot_loss: 4.834594266522617 - acc: 0.5308333333333334 - val_acc: 0.5555555555555556
Repeatition 1 Epoch 52 / 2000 
 - time: 7.860395908355713 - sq_loss: 3.2983916997909546 - tot_loss: 4.247879635333084 - acc: 0.5418333333333333 - val_acc: 0.5615615615615616
Repeatition 1 Epoch 53 / 2000 
 - time: 7.945122957229614 - sq_loss: 2.816762328147888 - tot_loss: 3.7455053924946697 - acc: 0.5511666666666667 - val_acc: 0.5755755755755756
Repeatition 1 Epoch 54 / 2000 
 - time: 7.926995754241943 - sq_loss: 2.405972182750702 - tot_loss: 3.3152672978889317 - acc: 0.5603333333333333 - val_acc: 0

Repeatition 1 Epoch 98 / 2000 
 - time: 7.940221309661865 - sq_loss: 0.00849355710670352 - tot_loss: 0.5076332266100507 - acc: 0.872 - val_acc: 0.8648648648648649
Repeatition 1 Epoch 99 / 2000 
 - time: 7.963046550750732 - sq_loss: 0.007968120044097304 - tot_loss: 0.5021138665520993 - acc: 0.8735 - val_acc: 0.8698698698698699
Repeatition 1 Epoch 100 / 2000 
 - time: 7.98270583152771 - sq_loss: 0.0074907951056957245 - tot_loss: 0.4967367979566916 - acc: 0.8763333333333333 - val_acc: 0.8708708708708709
Repeatition 1 Epoch 101 / 2000 
 - time: 7.976434946060181 - sq_loss: 0.007095691980794072 - tot_loss: 0.4915345300993067 - acc: 0.8785 - val_acc: 0.8718718718718719
Repeatition 1 Epoch 102 / 2000 
 - time: 7.98732328414917 - sq_loss: 0.0067409907933324575 - tot_loss: 0.4864661484840326 - acc: 0.8796666666666667 - val_acc: 0.8738738738738738
Repeatition 1 Epoch 103 / 2000 
 - time: 7.880375862121582 - sq_loss: 0.0064087758073583245 - tot_loss: 0.481507759120359 - acc: 0.8803333333333333 - 

Repeatition 1 Epoch 146 / 2000 
 - time: 8.634100675582886 - sq_loss: 0.00246989366132766 - tot_loss: 0.338151885573825 - acc: 0.9255 - val_acc: 0.924924924924925
Repeatition 1 Epoch 147 / 2000 
 - time: 8.407352924346924 - sq_loss: 0.0024392093473579735 - tot_loss: 0.3358544656344748 - acc: 0.926 - val_acc: 0.924924924924925
Repeatition 1 Epoch 148 / 2000 
 - time: 8.51976227760315 - sq_loss: 0.002408733416814357 - tot_loss: 0.33358713192938017 - acc: 0.9263333333333333 - val_acc: 0.924924924924925
Repeatition 1 Epoch 149 / 2000 
 - time: 8.466452360153198 - sq_loss: 0.0023850765137467533 - tot_loss: 0.3313597428608773 - acc: 0.9266666666666666 - val_acc: 0.9259259259259259
Repeatition 1 Epoch 150 / 2000 
 - time: 8.463392972946167 - sq_loss: 0.002351255825487897 - tot_loss: 0.32915088795234626 - acc: 0.928 - val_acc: 0.9259259259259259
Repeatition 1 Epoch 151 / 2000 
 - time: 8.519816875457764 - sq_loss: 0.0023255503037944436 - tot_loss: 0.326979327961817 - acc: 0.9285 - val_acc: 0.9

Repeatition 1 Epoch 193 / 2000 
 - time: 8.457952976226807 - sq_loss: 0.0016135491023305804 - tot_loss: 0.2566431730037948 - acc: 0.943 - val_acc: 0.934934934934935
Repeatition 1 Epoch 194 / 2000 
 - time: 8.482068061828613 - sq_loss: 0.0016023527132347226 - tot_loss: 0.2553522871176028 - acc: 0.9433333333333334 - val_acc: 0.9359359359359359
Repeatition 1 Epoch 195 / 2000 
 - time: 8.441439867019653 - sq_loss: 0.0015901301230769604 - tot_loss: 0.254069874052766 - acc: 0.9436666666666667 - val_acc: 0.9369369369369369
Repeatition 1 Epoch 196 / 2000 
 - time: 8.527271032333374 - sq_loss: 0.001577811926836148 - tot_loss: 0.2528059383703294 - acc: 0.9438333333333333 - val_acc: 0.9369369369369369
Repeatition 1 Epoch 197 / 2000 
 - time: 8.427865266799927 - sq_loss: 0.001565645943628624 - tot_loss: 0.25154996053861395 - acc: 0.944 - val_acc: 0.9369369369369369
Repeatition 1 Epoch 198 / 2000 
 - time: 8.44814944267273 - sq_loss: 0.0015585926303174347 - tot_loss: 0.25031683869019616 - acc: 0.94

...

Repeatition 1 Epoch 1374 / 2000 
 - time: 8.372097730636597 - sq_loss: 0.000253248872468248 - tot_loss: 0.042386454912821135 - acc: 0.9861666666666666 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1375 / 2000 
 - time: 8.441123485565186 - sq_loss: 0.000252451154665323 - tot_loss: 0.042355382535379245 - acc: 0.9861666666666666 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1376 / 2000 
 - time: 8.560253381729126 - sq_loss: 0.0002530350502638612 - tot_loss: 0.0423292942585249 - acc: 0.9863333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1377 / 2000 
 - time: 8.479035139083862 - sq_loss: 0.00025317749532405287 - tot_loss: 0.042302391311523024 - acc: 0.9863333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1378 / 2000 
 - time: 8.535341739654541 - sq_loss: 0.0002527966535126325 - tot_loss: 0.042277092609310785 - acc: 0.9863333333333333 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1379 / 2000 
 - time: 8.532796621322632 - sq_loss: 0.00025237250156351365 - to

Repeatition 1 Epoch 1422 / 2000 
 - time: 8.398999452590942 - sq_loss: 0.0002539787601563148 - tot_loss: 0.041114242404773904 - acc: 0.9865 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1423 / 2000 
 - time: 8.42888331413269 - sq_loss: 0.0002542229231039528 - tot_loss: 0.04108814827336573 - acc: 0.9865 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1424 / 2000 
 - time: 8.582035064697266 - sq_loss: 0.000253883954428602 - tot_loss: 0.04106324793860949 - acc: 0.9865 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1425 / 2000 
 - time: 8.449197769165039 - sq_loss: 0.0002535511157475412 - tot_loss: 0.04103635533638226 - acc: 0.9865 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1426 / 2000 
 - time: 8.478524446487427 - sq_loss: 0.0002519952613511123 - tot_loss: 0.04100921841218224 - acc: 0.9865 - val_acc: 0.980980980980981
Repeatition 1 Epoch 1427 / 2000 
 - time: 8.574228525161743 - sq_loss: 0.0002509952537366189 - tot_loss: 0.040983380582201794 - acc: 0.9865 - val_acc: 0.98098098

Repeatition 1 Epoch 1470 / 2000 
 - time: 8.44736123085022 - sq_loss: 0.00024859715267666616 - tot_loss: 0.0399169832910502 - acc: 0.9866666666666667 - val_acc: 0.9819819819819819
Repeatition 1 Epoch 1471 / 2000 
 - time: 8.53158164024353 - sq_loss: 0.00024788312657619826 - tot_loss: 0.03989390896440455 - acc: 0.9866666666666667 - val_acc: 0.9819819819819819
Repeatition 1 Epoch 1472 / 2000 
 - time: 8.547731399536133 - sq_loss: 0.0002474893699400127 - tot_loss: 0.03986929104949013 - acc: 0.9866666666666667 - val_acc: 0.9819819819819819
Repeatition 1 Epoch 1473 / 2000 
 - time: 8.576842308044434 - sq_loss: 0.0002479051545378752 - tot_loss: 0.03984771685427404 - acc: 0.9866666666666667 - val_acc: 0.9819819819819819
Repeatition 1 Epoch 1474 / 2000 
 - time: 8.503556728363037 - sq_loss: 0.000247900425165426 - tot_loss: 0.03982194651766804 - acc: 0.9866666666666667 - val_acc: 0.9819819819819819
Repeatition 1 Epoch 1475 / 2000 
 - time: 8.534497261047363 - sq_loss: 0.0002487510209903121 - to

Repeatition 1 Epoch 1516 / 2000 
 - time: 8.460341691970825 - sq_loss: 0.00024980319722089916 - tot_loss: 0.03884296559176619 - acc: 0.9871666666666666 - val_acc: 0.9819819819819819
Repeatition 1 Epoch 1517 / 2000 
 - time: 8.558479070663452 - sq_loss: 0.00024903945814003237 - tot_loss: 0.0388223446556367 - acc: 0.9871666666666666 - val_acc: 0.9819819819819819
Repeatition 1 Epoch 1518 / 2000 
 - time: 8.45402216911316 - sq_loss: 0.0002488305472070351 - tot_loss: 0.03879927233042508 - acc: 0.9871666666666666 - val_acc: 0.9819819819819819
Repeatition 1 Epoch 1519 / 2000 
 - time: 8.55991792678833 - sq_loss: 0.0002489789039827883 - tot_loss: 0.038776773726567626 - acc: 0.9871666666666666 - val_acc: 0.9819819819819819
Repeatition 1 Epoch 1520 / 2000 
 - time: 8.528470039367676 - sq_loss: 0.00024987464712467045 - tot_loss: 0.03875537488293048 - acc: 0.9871666666666666 - val_acc: 0.9819819819819819
Repeatition 1 Epoch 1521 / 2000 
 - time: 8.450182676315308 - sq_loss: 0.00025048362658708356 

Repeatition 1 Epoch 1562 / 2000 
 - time: 8.475966691970825 - sq_loss: 0.00023509584934799932 - tot_loss: 0.037822277919030964 - acc: 0.9878333333333333 - val_acc: 0.9819819819819819
Repeatition 1 Epoch 1563 / 2000 
 - time: 8.441647052764893 - sq_loss: 0.00023486099962610751 - tot_loss: 0.037800069810782586 - acc: 0.9878333333333333 - val_acc: 0.9819819819819819
Repeatition 1 Epoch 1564 / 2000 
 - time: 8.52331256866455 - sq_loss: 0.00023552047423436306 - tot_loss: 0.03778076055077691 - acc: 0.9878333333333333 - val_acc: 0.9819819819819819
Repeatition 1 Epoch 1565 / 2000 
 - time: 8.525238037109375 - sq_loss: 0.00023574180886498652 - tot_loss: 0.03775872260212054 - acc: 0.9878333333333333 - val_acc: 0.9819819819819819
Repeatition 1 Epoch 1566 / 2000 
 - time: 8.488416194915771 - sq_loss: 0.00023393507945002057 - tot_loss: 0.037735391846717904 - acc: 0.9878333333333333 - val_acc: 0.9819819819819819
Repeatition 1 Epoch 1567 / 2000 
 - time: 8.516453504562378 - sq_loss: 0.000233747996389